# Лабораторная работа №2
**"Непрерывные выпукло-вогнутые игры"**

**Выполнил: Александров А. Н., ИУ8-104**

**Вариант: 1**

## Задание
Функция ядра имеет вид:

$$H(x, y) = ax^2 + by^2 + cxy + dx + ey,$$

где:

|  a |   b  |   c  |   d  |   e  |
|:--:|:----:|:----:|:----:|:----:|
| -5 | 5/12 | 10/3 | -2/3 | -4/3 |

Найти оптимальные стратегии непрерывной выпукло-вогнутой антагонистической игры аналитическим и численным методами.

In [286]:
import logging

import sympy
from sympy import N, Eq, abc
# To represent multiple expressions in output of single cell.
from IPython.display import display

from game_theory.utils.continuous_games.convex_concave.numeric import NumericMethod

logging.basicConfig(level=logging.INFO, format='%(message)s')

In [287]:
ROUND_CONST: int = 3
A, B, C, D, E = (
    -5,
    5 / 12,
    10 / 3,
    -2 / 3,
    -4 / 3,
)

## 1. Аналитическое решение

In [288]:
# Входные параметры: коэффициенты функции ядра.
a, b, c, d, e = sympy.var(("a", "b", "c", "d", "e"))
# Переменные.
x, y = sympy.symbols(("x", "y"))

# Задание функции ядра. 
kernel_func = sympy.Lambda(
    (x, y),
    a * x ** 2 + b * y ** 2 + c * x * y + d * x + e * y,
)
kernel_func_subs = kernel_func.subs({a: A, b: B, c: C, d: D, e: E})

display(kernel_func)
N(kernel_func_subs, ROUND_CONST)

Lambda((x, y), a*x**2 + b*y**2 + c*x*y + d*x + e*y)

Lambda((x, y), -5.0*x**2 + 3.33*x*y - 0.667*x + 0.417*y**2 - 1.33*y)

In [289]:
kernel_xx = sympy.diff(kernel_func(x, y), x, 2, evaluate=False)
kernel_xx_eval = kernel_xx.doit()
kernel_xx_subs = kernel_xx_eval.subs({a: A, b: B, c: C, d: D, e: E})

display(Eq(kernel_xx, kernel_xx_eval))
N(Eq(kernel_xx_eval, kernel_xx_subs), ROUND_CONST)

Eq(Derivative(a*x**2 + b*y**2 + c*x*y + d*x + e*y, (x, 2)), 2*a)

Eq(2.0*a, -10.0)

In [290]:
kernel_yy = sympy.diff(kernel_func(x, y), y, 2, evaluate=False)
kernel_yy_eval = kernel_yy.doit()
kernel_yy_subs = kernel_yy_eval.subs({a: A, b: B, c: C, d: D, e: E})

display(Eq(kernel_yy, kernel_yy_eval))
N(Eq(kernel_yy_eval, kernel_yy_subs), ROUND_CONST)

Eq(Derivative(a*x**2 + b*y**2 + c*x*y + d*x + e*y, (y, 2)), 2*b)

Eq(2.0*b, 0.833)

In [291]:
is_convex_concave: bool = kernel_xx_subs < 0 < kernel_yy_subs
assert is_convex_concave, (
    "Игра не является выпукло-вогнутой, т.к. для функции ядра одновременно не выполняется оба условия: \n"
    f"H_xx = {2 * a:.2f} < 0 и H_yy = {2 * b:.2f} > 0"
)

Для нахождения оптимальных стратегий найдем производные функции ядра по каждой переменной

In [292]:
# Производная по x.
kernel_x = sympy.diff(kernel_func(x, y), x, evaluate=False)
kernel_x_eval = kernel_x.doit()
kernel_x_subs = kernel_x_eval.subs({a: A, b: B, c: C, d: D, e: E})
# Производная по y.
kernel_y = sympy.diff(kernel_func(x, y), y, evaluate=False)
kernel_y_eval = kernel_y.doit()
kernel_y_subs = kernel_y_eval.subs({a: A, b: B, c: C, d: D, e: E})

In [293]:
display(Eq(kernel_x, kernel_x_eval))
N(Eq(kernel_x_eval, kernel_x_subs), ROUND_CONST)

Eq(Derivative(a*x**2 + b*y**2 + c*x*y + d*x + e*y, x), 2*a*x + c*y + d)

Eq(2.0*a*x + c*y + d, -10.0*x + 3.33*y - 0.667)

In [294]:
display(Eq(kernel_y, kernel_y_eval))
N(Eq(kernel_y_eval, kernel_y_subs), ROUND_CONST)

Eq(Derivative(a*x**2 + b*y**2 + c*x*y + d*x + e*y, y), 2*b*y + c*x + e)

Eq(2.0*b*y + c*x + e, 3.33*x + 0.833*y - 1.33)

После приравнивания производных к нулю получим

In [295]:
# Выражаем решение производной через x.
zero_kernel_x, = sympy.solve(Eq(kernel_x_eval, 0), x)
zero_kernel_x_subs = zero_kernel_x.subs({a: A, b: B, c: C, d: D, e: E})
# Выражаем решение производной через y.
zero_kernel_y, = sympy.solve(Eq(kernel_y_eval, 0), y)
zero_kernel_y_subs = zero_kernel_y.subs({a: A, b: B, c: C, d: D, e: E})

In [296]:
display(Eq(x, zero_kernel_x))
display(N(Eq(x, zero_kernel_x_subs), ROUND_CONST))

Eq(x, (-c*y - d)/(2*a))

Eq(x, 0.333*y - 0.0667)

In [297]:
display(Eq(y, zero_kernel_y))
display(N(Eq(y, zero_kernel_y_subs), ROUND_CONST))

Eq(y, (-c*x - e)/(2*b))

Eq(y, 1.6 - 4.0*x)

Учитывая, что $х,y \ge 0$, для оптимальных стратегий имеем:

In [298]:
# Кусочно заданная функция относительно y.
psi_y = sympy.Piecewise(
    (zero_kernel_x, y >= -d / c),
    (0, y < -d / c)
)
psi_y_subs = psi_y.subs({a: A, b: B, c: C, d: D, e: E})
# Кусочно заданная функция относительно x.
phi_x = sympy.Piecewise(
    (zero_kernel_y, x <= -e / c),
    (0, x > -e / c)
)
phi_x_subs = phi_x.subs({a: A, b: B, c: C, d: D, e: E})

In [299]:
display(Eq(abc.psi, psi_y))
display(N(Eq(abc.psi, psi_y_subs), ROUND_CONST))

Eq(psi, Piecewise(((-c*y - d)/(2*a), y >= -d/c), (0, True)))

Eq(psi, Piecewise((0.333*y - 0.0667, y >= 0.2), (0, True)))

In [300]:
display(Eq(abc.phi, phi_x))
display(N(Eq(abc.phi, phi_x_subs), ROUND_CONST))

Eq(phi, Piecewise(((-c*x - e)/(2*b), x <= -e/c), (0, True)))

Eq(phi, Piecewise((1.6 - 4.0*x, x <= 0.4), (0, True)))

In [301]:
optimal_solution = sympy.solve((
    Eq(x, zero_kernel_x_subs),
    Eq(y, zero_kernel_y_subs),
))
x_opt, y_opt = optimal_solution.values()

saddle_point_value = float(kernel_func_subs(x_opt, y_opt))
print(f"Решение игры: \n"
      f"H({x_opt:.2f}, {y_opt:.2f}) = {saddle_point_value:.2f}")

Решение игры: 
H(0.20, 0.80) = -0.60


## 2. Численное решение
Для решения игры с непрерывным ядром используем метод аппроксимации функции выигрышей на сетке. 

In [302]:
numeric_method = NumericMethod(kernel_func_subs)

In [303]:
x_opt, y_opt, game_price_estimate = numeric_method.solve()

N = 2 (шаг: 0.500)
+-----------------------------------------------------------+
|                Таблица стратегий (игрока А)               |
+----------------+--------+--------+--------+---------------+
|   Стратегии    |   b1   |   b2   |   b3   | MIN выигрыш A |
+----------------+--------+--------+--------+---------------+
|       a1       | 0.000  | -0.562 | -0.917 |     -0.917    |
|       a2       | -1.583 | -1.312 | -0.833 |     -1.583    |
|       a3       | -5.667 | -4.562 | -3.250 |     -5.667    |
| MAX проигрыш B | 0.000  | -0.562 | -0.833 |               |
+----------------+--------+--------+--------+---------------+
Седловой точки нет. Решение методом Брауна-Робинсон:
x = 0.000; y = 0.500; H = -0.872

N = 3 (шаг: 0.333)
+--------------------------------------------------------------------+
|                    Таблица стратегий (игрока А)                    |
+----------------+--------+--------+--------+--------+---------------+
|   Стратегии    |   b1   |   b2   |   b3 

In [304]:
print(f"Решение игры: \n"
      f"x ≈ {x_opt:.2f}, y ≈ {y_opt:.2f}; H ≈ {game_price_estimate:.2f}")

Решение игры: 
x ≈ 0.22, y ≈ 0.73; H ≈ -0.63
